In [2]:
import http.server
import datetime
import socketserver
import threading
import subprocess
import time
import json
import requests

In [ ]:
import requests
import json

# Config Meta App + Instagram Variables
ig_user_id = "17841477150087921"
username = "smartinezg235"
app_id = "1082994883880852"
app_secret = "6e025991c9d5a0e9aed04cfbbed7cc89"
user_access_token = "EAAPYZBmVY55QBPlDZBUGGYAXtKlU1e2MyUXlvHonBfaPsl0WPnymbhBXTyZBvC6cZCEa9a5xr18y39lQSewwkL0zgUcSRfefnWpuKhEtc2cPWBcllrUFULFaPxDBZBY8rfnYAbbd2WSXT2va1ON8IiBwU1ZBnpUpDdR9dVOoNTyaWDV686ZBY9NgSXHsZAtEXnnJ2IzvB6xAZCa7w0o9evwH3LwyGnZAXAjOiF"

# Get Long Access Token
# https://graph.facebook.com/v17.0/oauth/access_token
# &grant_type=fb_exchange_token
# &client_id={app_id}
# &client_secret={app_secret}
# &fb_exchange_token={user_access_token}

url = f"https://graph.facebook.com/v17.0/oauth/access_token?grant_type=fb_exchange_token&client_id={app_id}&client_secret={app_secret}&fb_exchange_token={user_access_token}"
response = requests.get(url)
#print(response.content)
long_access_token = response.json()["access_token"]
long_access_token

'EAAPYZBmVY55QBPmwdctbE4j7CKfrI7CGLEm5kshDoZAgNaMNJDGZB1jEOKZCZA4gwM2jFUwEMfVdMmh538McGfcZADhSdhXuLba321sFtrHQ9TVMqgCGMX4ZChZAXAXg1dRuZBRnwnU7zUbnvpHvNeswsqZA4eeDaRiDGOfOaSyS2xtCiaIqfQs8AmM9ooVIG9evHX'

In [ ]:
long_access_token = 'EAAPYZBmVY55QBPmwdctbE4j7CKfrI7CGLEm5kshDoZAgNaMNJDGZB1jEOKZCZA4gwM2jFUwEMfVdMmh538McGfcZADhSdhXuLba321sFtrHQ9TVMqgCGMX4ZChZAXAXg1dRuZBRnwnU7zUbnvpHvNeswsqZA4eeDaRiDGOfOaSyS2xtCiaIqfQs8AmM9ooVIG9evHX'

### here i open a port locally so instagram can get my image

In [9]:

#Start a HTTP Server
PORT = 8080
Handler = http.server.SimpleHTTPRequestHandler
httpd = socketserver.TCPServer(("", PORT), Handler)
thread = threading.Thread(target=httpd.serve_forever)
thread.start()

127.0.0.1 - - [22/Sep/2025 00:20:45] "GET //media/uploads/WhatsApp_Image_2025-09-12_at_2_zCJQR6L.40.08_PM.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 00:20:45] code 404, message File not found
127.0.0.1 - - [22/Sep/2025 00:20:45] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Sep/2025 00:24:00] code 404, message File not found
127.0.0.1 - - [22/Sep/2025 00:24:00] "GET /robots.txt HTTP/1.1" 404 -
127.0.0.1 - - [22/Sep/2025 00:24:00] "GET /media/uploads/WhatsApp_Image_2025-09-12_at_2_JFcVPQO.40.08_PM.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 00:24:29] "GET /media/uploads/download_Gybqm9I.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 00:25:46] "GET /media/uploads/images.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2025 00:29:05] "GET /media/uploads/images_8PGpksj.jpg HTTP/1.1" 200 -


In [10]:
ngrok = subprocess.Popen(["ngrok", "http", str(PORT)], stdout=subprocess.PIPE)

# Give ngrok time to start up
time.sleep(2)

In [ ]:
#Get the public URL from ngrok
resp = requests.get("http://localhost:4040/api/tunnels")
public_url = resp.json()["tunnels"][0]["public_url"]

#Now you can use the public URL to access your local images
image_url = public_url + "/images.jpg"
print(image_url)



https://b6ef815b23d1.ngrok-free.app/images.jpg


In [ ]:

ngrok.terminate()
httpd.shutdown()

: 

In [ ]:
caption = "test para la clase"

url = f"https://graph.facebook.com/v17.0/{ig_user_id}/media"
payload = {
    "image_url" : image_url,
    "caption" : caption,
    "user_tags" : '[{"username":"mountainbetweenus", "x":0.5, "y":0.5}]',
    "collaborators" : '["mountainbetweenus"]',
    "access_token" : long_access_token,
}

#media_type" : "STORIES"
response = requests.post(url, params=payload)
data = response.json()
print(json.dumps(data, indent=4))
creation_id = data["id"]
print(f"creation_id : {creation_id}")

In [ ]:

#Get limit of publish posts allowes
url = f"https://graph.facebook.com/v17.0/{ig_user_id}/content_publishing_limit"
payload = {
  "fields": "quota_usage, config",
  "since": datetime.datetime(2025, 1, 12, 7, 30).timestamp(),
  "access_token": long_access_token
}
response = requests.get(url, params=payload)
metadata = response.json()
current_limit = metadata["data"][0]["quota_usage"]
quota_total = metadata["data"][0]["config"]["quota_total"]
print(json.dumps(metadata, indent=4))
print(f"You have used {current_limit} out of {quota_total} in a rolling 24-hour period")